In [19]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [20]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 8)
        self.fc2 = torch.nn.Linear(8, 8)
        self.fc3 = torch.nn.Linear(8, 1)

    def forward(self, x):
        y1 = torch.nn.functional.relu(self.fc1(x))
        y2 = torch.nn.functional.relu(self.fc2(y1))
        y3 = self.fc3(y2)
        return y3

In [21]:
model = MyModel()
model.load_state_dict(torch.load('model_file.pkl'))

<All keys matched successfully>

In [22]:
model.state_dict()
print(list(model.named_parameters())[2][1].data)

tensor([[ 0.1398,  0.1233, -0.2072, -0.1857,  0.2394, -0.0824, -0.2259, -0.0365],
        [-0.1383,  0.3103, -0.2891,  0.0753, -0.3075,  0.0308, -0.2461, -0.0866],
        [-0.6201,  0.1185,  0.0770,  0.3261, -0.2686,  0.3532,  0.2687, -0.4672],
        [-0.3240, -0.1424, -0.0219, -0.0521, -0.5631,  0.0064,  0.3645, -0.3428],
        [ 0.4360, -0.2108,  0.1669, -0.2691, -0.6525,  0.0222,  0.6670, -0.7854],
        [-0.0507, -0.2931,  0.2365,  0.1197,  0.0378, -0.0378, -0.3231, -0.2601],
        [-0.2656,  0.1492,  0.1287, -0.1474, -0.3995,  0.3117,  0.3375, -0.4090],
        [-0.3508, -0.2187, -0.0707,  0.0504, -0.2874, -0.0318, -0.1132,  0.2031]])


In [23]:
model.fc1.weight.shape

torch.Size([8, 1])

In [24]:
l = torch.zeros((1,1))
u = torch.ones((1,1))
w = model.fc1.weight

In [25]:
def fc_interval(l, u, w, b):
    s = (w>=0)*1
    w1 = s*w    
    w2 = (1-s)*w
    
    b = b.reshape(-1, 1)
    
    l_new = w1@l + w2@u + b
    u_new = w2@l + w1@u + b
    
    return l_new, u_new
    
def relu_interval(l, u):
    l_new = torch.zeros(l.shape)
    u_new = u.clone()
    
    indices = (l>0)
    l_new[indices] = l[indices]
    
    indices = (u<0)
    u_new[indices] = 0

    return l_new, u_new

In [26]:
n = 3
l = torch.zeros((1,1))
u = torch.ones((1,1))*0.5
for i in range(n):
    l, u = fc_interval(l, u, list(model.named_parameters())[2*i][1].data, list(model.named_parameters())[2*i+1][1].data)
    if i!=n-1:
        l, u = relu_interval(l, u)
print(l, u)
n = 3
l = torch.ones((1,1))*0.999
u = torch.ones((1,1))
for i in range(n):
    l, u = fc_interval(l, u, list(model.named_parameters())[2*i][1].data, list(model.named_parameters())[2*i+1][1].data)
    if i!=n-1:
        l, u = relu_interval(l, u)
print(l, u)

tensor([[-0.1450]]) tensor([[0.6428]])
tensor([[0.8621]]) tensor([[0.8652]])


In [27]:
n = 3
# l = torch.zeros((1,1))
# u = torch.ones((1,1))*0.5

partitions = 200
eps = 6/partitions


l1 = 100
u1 = -100

for p in range(partitions):
    l = torch.ones((1,1))*eps*p    
    u = torch.ones((1,1))*eps*(p+1)
    
    for i in range(n):
        l, u = fc_interval(l, u, list(model.named_parameters())[2*i][1].data, list(model.named_parameters())[2*i+1][1].data)
        if i!=n-1:
            l, u = relu_interval(l, u)
            
#     print(l, u)
    l1 = min(l1, l)
    u1 = max(u1, u)
    
print(l1, u1)

tensor([[-1.0726]]) tensor([[1.1007]])


In [28]:
import math

In [29]:
model(torch.tensor([4.8]))

tensor([-0.9937], grad_fn=<AddBackward0>)

In [30]:
def fc_d_interval(l, u, w, b):
#     L, U are matrices now. L_{i, j} is the derivative of x_i wrt I_j
    s = (w>=0)*1
    w1 = s*w    
    w2 = (1-s)*w
    
    l_new = w1@l + w2@u
    u_new = w2@l + w1@u
    
    return l_new, u_new

In [31]:
def relu_d_interval(l, u, L, U):
#     l, u : n X 1
#     L, U : n X d
#     L_new, U_new : n X d
    L_temp_new = torch.zeros((l.shape))
    U_temp_new = torch.ones((l.shape))
    
    indices = (l>0)
    L_temp_new[indices] = 1
    
    indices = (u<0)
    U_temp_new[indices] = 0

    L_new = L * L_temp_new.view(-1,1)
    U_new = U * U_temp_new.view(-1,1)
    return L_new, U_new

In [32]:
X = torch.tensor([[1, 2, 3],
                  [4, 5, 6]], dtype=torch.float)

y = torch.tensor([2, 3], dtype=torch.float)
# Element-wise multiplication of X and y
result = X * y.view(-1, 1)
print(result)

tensor([[ 2.,  4.,  6.],
        [12., 15., 18.]])


In [33]:
X = torch.tensor([[1, 2, 3],
                  [4, 5, 6]], dtype=torch.float)

y = torch.tensor([2, 3], dtype=torch.float)

# Create a diagonal matrix from y
diag_y = torch.diag(y)
print(diag_y)
# Multiply X by the diagonal matrix diag_y
result = torch.mm(diag_y, X)

print(result)

tensor([[2., 0.],
        [0., 3.]])
tensor([[ 2.,  4.,  6.],
        [12., 15., 18.]])


In [34]:
n = 3

partitions = 1
eps = 1/partitions


# l1 = 100
# u1 = -100

for p in range(partitions):
    l = torch.ones((1,1))*eps*p    
    u = torch.ones((1,1))*eps*(p+1)
    L = torch.eye(1)
    U = torch.eye(1)
    
    for i in range(n):
        w = list(model.named_parameters())[2*i][1].data
        b = list(model.named_parameters())[2*i+1][1].data
        l, u = fc_interval(l, u, w, b)
        L, U = fc_d_interval(L, U, w, b)
        if i!=n-1:
            L, U = relu_d_interval(l, u, L, U)
            l, u = relu_interval(l, u)

print(l, u, L, U)

tensor([[-0.3916]]) tensor([[1.2869]]) tensor([[-0.2001]]) tensor([[1.3707]])
